In [20]:
# Import necessary libraries

import numpy as np
import pandas as pd
import pandapower as pp
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pp_heig_simulation as pp_sim
import pickle
import re 



Import pickle ("trey_net_student.p")

In [21]:
# Load the pandapower network from the pickle file
net_trey = pp.from_pickle("input-data/trey_net_student.p")

# quick inspection
print(type(net_trey))
print(net_trey)

<class 'pandapower.auxiliary.pandapowerNet'>
This pandapower network includes the following parameter tables:
   - bus (13 elements)
   - load (12 elements)
   - ext_grid (1 element)
   - line (11 elements)
   - trafo (1 element)


Design of network 

In [22]:
# Bus 
net_trey.bus


,name,type,zone,in_service,geo
0,STMT003438,PQ,Trafo,True,NaN
1,CDBT004764,PQ,North,True,NaN
2,CDBT003746,PQ,North,True,NaN
3,CDBT004760,PQ,North,True,NaN
4,CDBT012139,PQ,North,True,NaN
5,CDBT900784,PQ,North,True,NaN
6,CDBT901452,PQ,South,True,NaN
7,CDBT004774,PQ,South,True,NaN
8,CDBT901604,PQ,South,True,NaN
9,CDBT016055,PQ,South,True,NaN


In [23]:
# load
net_trey.load


,name,bus,const_z_p_percent,const_z_q_percent,const_i_p_percent,const_i_q_percent,sn_mva,in_service,type
0,STMT003438,0,0.0,0.0,0.0,0.0,NaN,True,wye
1,CDBT004764,1,0.0,0.0,0.0,0.0,NaN,True,wye
2,CDBT003746,2,0.0,0.0,0.0,0.0,NaN,True,wye
3,CDBT004760,3,0.0,0.0,0.0,0.0,NaN,True,wye
4,CDBT012139,4,0.0,0.0,0.0,0.0,NaN,True,wye
5,CDBT900784,5,0.0,0.0,0.0,0.0,NaN,True,wye
6,CDBT901452,6,0.0,0.0,0.0,0.0,NaN,True,wye
7,CDBT004774,7,0.0,0.0,0.0,0.0,NaN,True,wye
8,CDBT901604,8,0.0,0.0,0.0,0.0,NaN,True,wye
9,CDBT016055,9,0.0,0.0,0.0,0.0,NaN,True,wye


In [24]:
# ext_grid
net_trey.ext_grid   

,name,vm_pu,va_degree,slack_weight,s_sc_max_mva,s_sc_min_mva,rx_min,rx_max,r0x0_max,x0x_max,in_service
0,ext_grid_0,1.0,0.0,1.0,100.0,None,None,0.1,2.0,2.0,True


In [25]:
# Line
net_trey.line



,name,from_bus,to_bus,g_us_per_km,df,std_type,in_service,geo
0,GKN3x150_150,0,1,0.0,1.0,None,True,NaN
1,GKN3x150_150_2,1,2,0.0,1.0,None,True,NaN
2,GKN3X95_95,1,3,0.0,1.0,None,True,NaN
3,GKT3X50_50,3,4,0.0,1.0,None,True,NaN
4,GKT3X50_50_2,4,5,0.0,1.0,None,True,NaN
5,GKN3x150_150_3,0,6,0.0,1.0,None,True,NaN
6,GKN3X95_95_2,6,7,0.0,1.0,None,True,NaN
7,GKN3x150_150_4,7,8,0.0,1.0,None,True,NaN
8,GKN3X50_50,7,9,0.0,1.0,None,True,NaN
9,GKN3X50_50_2,6,10,0.0,1.0,None,True,NaN


In [ ]:
# Use the datasheet to create a dictionary with line parameters 

## Create new column "cable_type" by removing the unwanted value (_x) use to differentiate each line from "name" with regex
# re.sub replaces the matches with the given substitute string
# First argument: regex pattern
# Second argument: substitute string
# Third argument: input string

net_trey.line.insert(
    0, "length_km", [0.180, 0.1, 0.115, 0.08, 0.09, 0.135, 0.1, 0.205, 0.085, 0.255, 0.340]
)
net_trey.line["cable_type"] = net_trey.line.apply(
    lambda x: re.sub(r"_[0-9]$", "", x["name"]), axis=1
)
 
## Create parameter table for cable types
cable_types = pd.DataFrame(
    {
        "cable_type": ["GKN3x150_150", "GKN3X95_95", "GKN3X50_50", "GKN3X240_240", "GKT3X50_50"],
        "r_ohm_per_km": [0.124, 0.193, 0.387, 0.0754, 0.387],
        "x_ohm_per_km": [0.07, 0.07, 0.07, 0.07, 0.07],
        "c_uf_per_km": [349, 338, 298, 346, 298],
        "max_i_ka": [0.4, 0.252, 0.17, 0.512, 0.17],
    }
)
 
## Merge net_trey.line with cable_types on "cable_type"
net_trey.line = net_trey.line.merge(cable_types, on="cable_type", how="left")
 
## Show updated net_trey.line
net_trey.line
 
# les valeurs de max_i_ka sont les valeur pour des cables enterrés. en effet le reseau suisse est majoriterrement souterrain en zone urbaine.


ValueError: All arrays must be of the same length

In [ ]:
# tranformer
net_trey.trafo

,name,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,i0_percent,...,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,parallel,std_type,df,in_service,tap_changer_type
0,STMT003438,12,0,0.63,18.3,0.42,4.0,0.42,0.65,1.8,...,-3.0,3.0,2.0,0.0,-3.0,1,None,1.0,True,Ratio
